In [ ]:
import pandas as pd
import numpy as np
import pypsa

In [ ]:
l = []

In [ ]:
n = pypsa.Network()
n.import_from_netcdf(snakemake.input['network'])

In [ ]:
res = n.generators_t['p'].filter(regex='wind|pvplant')
res = res.loc[:, (res != 0.).any()]
res_opt = res.divide(n.generators['p_nom_opt'])
res_opt = res_opt.loc[:, (res_opt != 0.).any(axis=0)].dropna(axis=1)

In [ ]:
l.append(['general', 'Total system cost', n.objective])

In [ ]:
# Price per MWh delivered
l.append(['general', 'Cost per MWh delivered', n.objective / n.loads_t['p'].to_numpy().sum()])

In [ ]:
# Energy surplus factor
l.append(['general', 'Energy surplus factor', res.to_numpy().sum()/n.loads_t['p'].to_numpy().sum()])

In [ ]:
# Installed RES capacity
l.append(['general', 'Installed RES capacity', n.generators.filter(regex='wind|pvplant', axis=0)['p_nom_opt'].sum()])

In [ ]:
# costs

In [ ]:
t = getattr(n, 'generators')
t = (t['p_nom_opt'].clip(lower=0.)*t['capital_cost']).replace(0.,np.nan).dropna()

t = list(zip(['cost']*len(t), t.index, t.to_list()))
display(t)
l.extend(t)

In [ ]:
t = getattr(n, 'links')
t = (t['p_nom_opt'].clip(lower=0)*t['capital_cost']).replace(0., np.nan).dropna()

t = list(zip(['cost']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
t = getattr(n, 'stores')
t = (t['e_nom_opt'].clip(lower=0)*t['capital_cost']).replace(0., np.nan).dropna()

t = list(zip(['cost']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
g = n.generators
g = g.loc[g['marginal_cost'] > 0.]
t = (n.generators_t['p'][g.index]*g['marginal_cost']).sum()

t = list(zip(['cost']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
# capacity factors

In [ ]:
# Annual CF
t = res.to_numpy().sum()/(n.generators.filter(regex='wind|pvplant', axis=0)['p_nom_opt'].sum()*n.snapshots.shape[0])
display(t)

l.append(['capacity factor', 'RES annual CF', t])

In [ ]:
# Average annual CF
t = res_opt.to_numpy().mean()

display(t)
l.append(['capacity factor', 'RES average CF', t])

In [ ]:
c = 'stores'
t = getattr(n, c)
t_t = getattr(n, c+"_t")

t = t_t['e'].divide(t['e_nom_opt'].clip(lower=0.)).dropna(axis=1).mean()
t = list(zip(['capacity factor']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
c = 'links'
t = getattr(n, c)
t_t = getattr(n, c+"_t")

t = t_t['p0'].divide(t['p_nom_opt'].clip(lower=0.)).dropna(axis=1).mean()
t = list(zip(['capacity factor']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
c = 'generators'
t = getattr(n, c)
t_t = getattr(n, c+"_t")

t = t_t['p'].divide(t['p_nom_opt'].clip(lower=0.)).dropna(axis=1).mean()
t = list(zip(['capacity factor']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
# build capacities

In [ ]:
c = 'generators'
t = getattr(n, c)

t = t['p_nom_opt'].clip(lower=0.).replace(0., np.nan).dropna()
t = list(zip(['installed capacity']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
c = 'links'
t = getattr(n, c)
t = t['p_nom_opt'].clip(lower=0.).replace(0., np.nan).dropna()

t = list(zip(['installed capacity']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
c = 'stores'
t = getattr(n, c)
t = t['e_nom_opt'].clip(lower=0.).replace(0., np.nan).dropna()

t = list(zip(['installed capacity']*len(t), t.index, t.to_list()))
display(t)

l.extend(t)

In [ ]:
# min electricity generation
t = res.sum(axis=1).min()

l.append(['general', 'minimum RES generation', t])

In [ ]:
# max electricity generation
t = res.sum(axis=1).max()

l.append(['general', 'maximum RES generation', t])

In [ ]:
# Curtailed energy
real = res.sum(axis=1)
theo_max = n.generators_t['p_max_pu'].multiply(n.generators['p_nom_opt']).replace(0., np.nan).dropna(axis=1).sum(axis=1)
curtailed_energy = (theo_max-real).sum()

l.append(['general', 'curtailed generation', curtailed_energy])

In [ ]:
l = pd.DataFrame(l, columns=['category', 'subcategory', 'value'])
l = l.set_index(['category','subcategory'])

In [ ]:
l.to_csv(snakemake.output['results'], sep=';', quotechar='"')

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(30,30))
ax = plt.gca()

res = n.generators_t['p'].filter(regex='pvplant|wind',axis=1)
res = res.loc[:,(res != 0.).any()]

real = res.sum(axis=1)
theo_max = n.generators_t['p_max_pu'].multiply(n.generators['p_nom_opt'])[res.columns].sum(axis=1)
diff = real-theo_max

theo_max.name = "Available generation"
real.name = "Realised generation"
diff.name = "Curtailed"

theo_max.plot(ax=ax)
real.plot(ax=ax)
diff.plot(ax=ax)

plt.legend()

In [ ]:
if 'shipping' in n.name:
    n.links_t['p0'].filter(regex='ship convoy [0-9]* loading').sum(axis=1).plot(figsize=(30,10),
                                                                                title='ship loading')

In [ ]:
if 'shipping' in n.name:
    n.links_t['p0'].filter(regex='ship convoy [0-9]* unloading').sum(axis=1).plot(figsize=(30,10),
                                                                               title='ship unloading')

In [ ]:
if 'shipping' in n.name:
    n.stores_t['e'].filter(like='cargo').sum(axis=1).plot(figsize=(30,10),
                                                      title='ship cargo')

In [ ]:
cs = {c for c in n.links.index if "ship convoy" not in c}

In [ ]:
c1 = {'battery inverter (charging, exp)',
     'battery inverter (discharging, exp)',
     'seawater desalination (exp)'}
c2 = {'seawater desalination (exp)'}
c3 = cs - c1 - c2

In [ ]:
n.links_t['p0'].index = pd.to_datetime(n.links_t['p0'].index)

In [ ]:
for c in [c1,c2,c3]:
    n.links_t['p0'][c].rolling(window=6).mean().plot(figsize=(30,10))

In [ ]:
cs = {c for c in n.stores_t['e'].columns if 'cargo' not in c}

In [ ]:
c1 = {"hydrogen (g) storage tank (exp)",
         "hydrogen (g) demand buffer (imp)"}
c2 = {"clean water tank storage (exp)",
         "battery storage (exp)"}
c3 = cs-c1-c2

In [ ]:
for c in [c1,c2,c3]:
    n.stores_t['e'][c].plot(figsize=(30,20), title='Stores')